In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Install keras==3.1.1 before using model otherwise it will give batch shape error

In [ ]:
!pip install keras==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from collections import Counter
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Layer
from keras.layers import Attention, Input
import keras.backend as K
from tensorflow.keras.backend import squeeze,softmax,dot,expand_dims,tanh
from tensorflow.keras.backend import sum as sums
from keras.models import load_model

# For GPU testing

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("GPU device not found. Using CPU instead.")

# Load file for training here Label included in dataset

In [ ]:
def load_file(path):
    data = pd.read_csv(path)
    fields=['src_port', 'dst_port',
       'protocol','flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt',
       'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts',
       'init_bwd_win_byts', 'active_max', 'active_min', 'active_mean',
       'active_std', 'idle_max', 'idle_min', 'idle_mean', 'idle_std',
       'fwd_byts_b_avg', 'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_blk_rate_avg', 'fwd_seg_size_avg',
       'bwd_seg_size_avg', 'cwe_flag_count', 'subflow_fwd_pkts',
       'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts','Label']
    data=data[fields]
    return data

# Load file for testing here Label not included in fields

In [ ]:
def load_file1(path):
    data = pd.read_csv(path)
    fields=['src_port', 'dst_port',
       'protocol','flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt',
       'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts',
       'init_bwd_win_byts', 'active_max', 'active_min', 'active_mean',
       'active_std', 'idle_max', 'idle_min', 'idle_mean', 'idle_std',
       'fwd_byts_b_avg', 'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_blk_rate_avg', 'fwd_seg_size_avg',
       'bwd_seg_size_avg', 'cwe_flag_count', 'subflow_fwd_pkts',
       'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts']
    data=data[fields]
    return data

def load_file_old(path):
    data = pd.read_csv(path)
    fields=[' Source Port',' Destination Port',' Protocol',' Flow Duration','Flow Bytes/s',' Flow Packets/s',
        'Fwd Packets/s',' Bwd Packets/s',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',
       ' Fwd Packet Length Min',' Fwd Packet Length Mean', ' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std',
        ' Max Packet Length',' Min Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance',' Fwd Header Length',' Bwd Header Length',' Flow IAT Mean',
       ' Flow IAT Max',' Flow IAT Min',' Flow IAT Std','Fwd IAT Total',' Fwd IAT Max',' Fwd IAT Min',' Fwd IAT Mean',' Fwd IAT Std','Bwd IAT Total',' Bwd IAT Max',' Bwd IAT Min',
       ' Bwd IAT Mean',' Bwd IAT Std','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',
        ' ACK Flag Count',' URG Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size','Init_Win_bytes_forward',' Init_Win_bytes_backward',' Active Max',' Active Min',
        'Active Mean',' Active Std',' Idle Max',' Idle Min','Idle Mean', ' Idle Std','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk',
        ' Fwd Avg Bulk Rate','Bwd Avg Bulk Rate',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' CWE Flag Count','Subflow Fwd Packets',' Subflow Bwd Packets',' Subflow Fwd Bytes',
        ' Subflow Bwd Bytes']
    data=data[fields]
    return data

In [ ]:
labels = {
    '0': "Benign",
    '1': "Golden_eye",
    '2': "vsi",
    '3': "slow_http",
}

In [ ]:

def format_3d(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], X.shape[1], 1))

def preprocess_dataset(dataset):
    # Replace 'Infinity' with 0
    dataset = dataset.replace('Infinity', '0')
    dataset = dataset.replace(np.inf, '0')
    # Convert columns to numeric
    dataset['flow_pkts_s'] = pd.to_numeric(dataset['flow_pkts_s'])
    dataset['flow_byts_s'] = pd.to_numeric(dataset['flow_byts_s'].fillna(0))
    print(dataset['Label'].value_counts())
    label_encoder = LabelEncoder()
    label_encoder.fit(list(labels.values()))
    dataset['Label'] = label_encoder.transform(dataset['Label'])
    print(dataset['Label'].value_counts())
    return dataset

def preprocess_dataset1(dataset):
    # Replace 'Infinity' with 0
    dataset = dataset.replace('Infinity', '0')
    dataset = dataset.replace(np.inf, '0')
    # Convert columns to numeric
    dataset['flow_pkts_s'] = pd.to_numeric(dataset['flow_pkts_s'])
    dataset['flow_byts_s'] = pd.to_numeric(dataset['flow_byts_s'].fillna(0))
    return dataset

def preprocess_dataset_old(samples):
  samples = samples.replace('Infinity','0')
  samples = samples.replace(np.inf,0)
  #samples = samples.replace('nan','0')
  samples[' Flow Packets/s'] = pd.to_numeric(samples[' Flow Packets/s'])

  samples['Flow Bytes/s'] = samples['Flow Bytes/s'].fillna(0)
  samples['Flow Bytes/s'] = pd.to_numeric(samples['Flow Bytes/s'])
  return samples

def GraphPlot(hist,model):
    plt.plot(hist.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    print(model.metrics_names)

def train_test(X,y):
    X_train, X_test = X[:-11000], X[-11000:]
    y_train, y_test = y[:-11000], y[-11000:]
    return  X_train,X_test,y_train,y_test

In [ ]:

def save_model(model, name):
    # Save the entire model as a .keras file
    model.save(name + '.keras')
    print('Model Saved')

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = squeeze(tanh(dot(x, self.W) + self.b), axis=-1)
        at = softmax(et)
        at = expand_dims(at, axis=-1)
        output = x * at
        return sums(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def lstmModel(outputlen):
    base_model = Sequential()
    base_model.add(LSTM(units=50, return_sequences=True, input_shape=( 77,1))) #Changed here
    base_model.add(Dropout(0.3))
    base_model.add(LSTM(units=50, return_sequences=True))
    base_model.add(Dropout(0.3))
    base_model.add(AttentionLayer())
    base_model.add(Dense(units=4, activation='softmax'))
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return base_model

In [ ]:
data=load_file('/kaggle/input/traffic1/Labelled_traffic1.csv')

fdata=preprocess_dataset(data)
x=fdata.drop(columns=['Label']).values
x=format_3d(x)
y=fdata['Label']
y = to_categorical(y, num_classes=4)
print(y)

In [ ]:
# with strategy.scope():
X_train,X_test,y_train,y_test=train_test(x,y)
y = to_categorical(y, num_classes=4)
model=lstmModel(4)
history=model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
save_model(model,'Model')
loss, accuracy = model.evaluate(X_test, y_test)
# print(f'Loss on test set for {path}: {loss}')
# print(f'Accuracy on test set for {path}: {accuracy}')
GraphPlot(history,model)

# For testing trained model on Dataset

In [ ]:
test=load_file1('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/liveCapture.csv')

In [ ]:
model = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/ModelvsiLstm.keras', custom_objects={'AttentionLayer': AttentionLayer})

In [ ]:
processeddata=preprocess_dataset1(test)
Reshaped=format_3d(processeddata)
prediction=model.predict(processeddata)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step


In [ ]:
predictions = np.argmax(prediction, axis=1)
label_counts = Counter(predictions)
max_count = 0
# print("Counts of unique labels:")
for label, count in label_counts.items():
    tlabel=labels[str(label)]
    print(f"{tlabel}: {count}")
    if count > max_count:
        max_count = count
        max_model = tlabel

print(f"Model with maximum predictions: {max_model} ({max_count} predictions)")

Benign: 299
Model with maximum predictions: Benign (299 predictions)


# For source ip address of attacker

In [ ]:

fields=['src_port', 'dst_port',
       'protocol','flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt',
       'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts',
       'init_bwd_win_byts', 'active_max', 'active_min', 'active_mean',
       'active_std', 'idle_max', 'idle_min', 'idle_mean', 'idle_std',
       'fwd_byts_b_avg', 'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_blk_rate_avg', 'fwd_seg_size_avg',
       'bwd_seg_size_avg', 'cwe_flag_count', 'subflow_fwd_pkts',
       'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts']
labels = {
    '0': "Benign",
    '1': "Golden_eye",
    '2': "vsi",
    '3': "slow_http",
}

def detect_anomaly(csv_path, model):
    # Read data
    data = pd.read_csv(csv_path)
    source_ip = data['src_ip']
    data = data[fields]

    # Handle special values
    dataset = data.replace('Infinity', '0')
    dataset = dataset.replace(np.inf, '0')
    dataset['flow_pkts_s'] = pd.to_numeric(dataset['flow_pkts_s'])
    dataset['flow_byts_s'] = pd.to_numeric(dataset['flow_byts_s'].fillna(0))

    # Make predictions
    predictions = model.predict(data)
    predictions = np.argmax(predictions, axis=1)

    # Count predictions
    label_counts = Counter(predictions)
    max_count = 0
    max_model=''
    for label, count in label_counts.items():
        tlabel = labels[str(label)]
        print(f"{tlabel}: {count}")
        if count > max_count and tlabel != "Benign":
            max_count = count
            max_model = tlabel

    print(f"Model with maximum predictions: {max_model} ({max_count} predictions)")

    # Store anomalies
    unique_ips = set()  # To store unique IPs
    anomalies = []
    for i in range(len(predictions)):
        if predictions[i] != "Benign" and labels[str(predictions[i])] == max_model:
            ip = source_ip[i]
            if ip not in unique_ips:  # Check if IP is already seen
                unique_ips.add(ip)
                anomalies.append((ip, labels[str(predictions[i])]))

    # Print the results
    print('Source IP | Anomaly')
    print('===================')
    for ip, anomaly in anomalies:
        print(f"{ip} | {anomaly}")

In [ ]:
detect_anomaly('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/liveCapture.csv', model)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Benign: 299


UnboundLocalError: local variable 'max_model' referenced before assignment